In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import random
import os
from PIL import Image
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader, Dataset
from torchvision import datasets, transforms
from tqdm.notebook import tqdm
#from linformer import Linformer

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
%pip install einops
%pip install wandb

     |████████████████████████████████| 2.1MB 6.4MB/s 
     |████████████████████████████████| 163kB 26.7MB/s 
     |████████████████████████████████| 133kB 23.3MB/s 
     |████████████████████████████████| 102kB 10.0MB/s 
     |████████████████████████████████| 71kB 8.5MB/s 
  Created wheel for pathtools: filename=pathtools-0.1.2-cp37-none-any.whl size=8786 sha256=fe19aa9c36728e90a3b61aca7a02dad450e3362e0808831cd9ad5b935364c590
  Stored in directory: /root/.cache/pip/wheels/0b/04/79/c3b0c3a0266a3cb4376da31e5bfe8bba0c489246968a68e843
  Created wheel for subprocess32: filename=subprocess32-3.5.4-cp37-none-any.whl size=6489 sha256=c851781553ed5b8672a93a999277cfee9492cf0c57a66a7cb0b39263773256a5
  Stored in directory: /root/.cache/pip/wheels/68/39/1a/5e402bdfdf004af1786c8b853fd92f8c4a04f22aad179654d1
Successfully built pathtools subprocess32


In [4]:
%cd drive/MyDrive/Colab Notebooks/ViT

/content/drive/MyDrive/Colab Notebooks/ViT


In [9]:
from model.Cross_ViT import CrossViT
from dataloader import set_transforms
from utils.utils import seed_everything

seed_everything(42)

In [10]:
import torch.backends.cudnn as cudnn

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
use_cuda = torch.cuda.is_available()
cudnn.benchmark = True
print('Use CUDA:', use_cuda)

Use CUDA: True


In [11]:
config={
    "epochs": 20, 
    "batch_size": 64,
    "lr" : 3e-5,
    "gamma" : 0.7,
    "image_size" : 32,
    "patch_size" : 16,
    "num_classes" : 10,
    "dim" : 128,
    "depth" : 3,
    "heads" : 8,
    "mlp_dim" : 1024,
    "channels" : 3,
    "dropout" : 0.,
    "emb_dropout": 0.
    }

# Training settings
import wandb
wandb.init(config=config, project="Cifar10_ViT", entity='NoguNogu',name='CrossViT')

<IPython.core.display.Javascript object>

wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [12]:
trainloader, testloader = set_transforms(
    config['image_size'],
    config['batch_size'],
    name='Cifar10')._set_transforms()

In [17]:
model = CrossViT(
    image_size = config['image_size'],
    num_classes = config['num_classes'],
    sm_patch_size = 8,
    sm_dim = config['dim'],
    lg_dim = config['dim'],
    depth = config['depth'],
    dropout = config['dropout'],
    emb_dropout = config['emb_dropout']
    ).to(device)

In [18]:
from torchsummary import summary
summary(model, (3, 32, 32))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
         Rearrange-1              [-1, 16, 192]               0
            Linear-2              [-1, 16, 128]          24,704
           Dropout-3              [-1, 17, 128]               0
  Patch_Linear_emb-4              [-1, 17, 128]               0
         Rearrange-5               [-1, 4, 768]               0
            Linear-6               [-1, 4, 128]          98,432
           Dropout-7               [-1, 5, 128]               0
  Patch_Linear_emb-8               [-1, 5, 128]               0
         LayerNorm-9              [-1, 17, 128]             256
           Linear-10              [-1, 17, 512]          65,536
           Linear-11             [-1, 17, 1024]         131,072
          Softmax-12            [-1, 8, 17, 17]               0
           Linear-13              [-1, 17, 128]          65,664
          Dropout-14              [-1, 

In [19]:
# loss function
criterion = nn.CrossEntropyLoss()
# optimizer
optimizer = optim.Adam(model.parameters(), lr=config['lr'])
# scheduler
scheduler = StepLR(optimizer, step_size=1, gamma=config['gamma'])

In [20]:
# 3. Log gradients and model parameters
wandb.watch(model, log_freq=100)
for epoch in range(config['epochs']):
    epoch_loss = 0
    epoch_accuracy = 0

    for data, label in tqdm(trainloader):
        data = data.to(device)
        label = label.to(device)

        output = model(data)
        loss = criterion(output, label)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        acc = (output.argmax(dim=1) == label).float().mean()
        epoch_accuracy += acc / len(trainloader)
        epoch_loss += loss / len(trainloader)

    with torch.no_grad():
        epoch_val_accuracy = 0
        epoch_val_loss = 0
        for data, label in testloader:
            data = data.to(device)
            label = label.to(device)

            val_output = model(data)
            val_loss = criterion(val_output, label)

            acc = (val_output.argmax(dim=1) == label).float().mean()
            epoch_val_accuracy += acc / len(testloader)
            epoch_val_loss += val_loss / len(testloader)

    print(
        f"Epoch : {epoch+1} - loss : {epoch_loss:.4f} - acc: {epoch_accuracy:.4f} - val_loss : {epoch_val_loss:.4f} - val_acc: {epoch_val_accuracy:.4f}\n"
    )

    # 4. Log metrics to visualize performance
    wandb.log({
        "Epoch": epoch+1,
        "loss": epoch_loss,
        "acc": epoch_accuracy,
        "val_loss" : epoch_val_loss,
        "val_acc": epoch_val_accuracy
        })


Epoch : 1 - loss : 2.0774 - acc: 0.2233 - val_loss : 1.9658 - val_acc: 0.2726




Epoch : 2 - loss : 1.9292 - acc: 0.2875 - val_loss : 1.8487 - val_acc: 0.3198




Epoch : 3 - loss : 1.8385 - acc: 0.3218 - val_loss : 1.8118 - val_acc: 0.3393




Epoch : 4 - loss : 1.7855 - acc: 0.3482 - val_loss : 1.7523 - val_acc: 0.3691




Epoch : 5 - loss : 1.7388 - acc: 0.3637 - val_loss : 1.7191 - val_acc: 0.3777



...


Epoch : 16 - loss : 1.5236 - acc: 0.4502 - val_loss : 1.5337 - val_acc: 0.4520




Epoch : 17 - loss : 1.5150 - acc: 0.4569 - val_loss : 1.5253 - val_acc: 0.4516




Epoch : 18 - loss : 1.5065 - acc: 0.4583 - val_loss : 1.5315 - val_acc: 0.4528




Epoch : 19 - loss : 1.4885 - acc: 0.4619 - val_loss : 1.4958 - val_acc: 0.4696




Epoch : 20 - loss : 1.4831 - acc: 0.4671 - val_loss : 1.5100 - val_acc: 0.4556



In [15]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Thu May  6 12:05:32 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 465.19.01    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    30W / 250W |      2MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------